In [181]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from datetime import date, timedelta, datetime
from time import sleep
from urllib.error import HTTPError
import re
import json

team_names = {
    'ATL': 'ATL',
    'BOS': 'BOS',
    'BKN': 'BRK',
    'CHA': 'CHO',
    'CHI': 'CHI',
    'CLE': 'CLE',
    'DAL': 'DAL',
    'DEN': 'DEN',
    'DET': 'DET',
    'HOU': 'HOU',
    'IND': 'IND',
    'GS': 'GSW',
    'LAC': 'LAC',
    'LAL': 'LAL',
    'MEM': 'MEM',
    'MIA': 'MIA',
    'MIL': 'MIL',
    'MIN': 'MIN',
    'NO': 'NOP',
    'NY': 'NYK',
    'OKC': 'OKC',
    'ORL': 'ORL',
    'PHI': 'PHI',
    'PHX': 'PHO',
    'POR': 'POR',
    'SAC': 'SAC',
    'SA': 'SAS',
    'TOR': 'TOR',
    'UTAH': 'UTA',
    'WSH': 'WAS',
}


def process_game(soup, team):

    return


def scrape(game_id, game_number):
    gameid = game_id + game_number
    print(gameid)
    gameid_str = '00%d' % gameid
    try:
        url = 'http://stats.inpredictable.com/nba/wpBox.php?gid=' + gameid_str
        url_page = urlopen(url)
    except HTTPError:
        sleep(10)
        url_page = urlopen(url)
    soup = BeautifulSoup(url_page, "html.parser")
    score_data = str(soup.find("div", {"id": "score"}))

    ### GET home, away, date
    away = soup.find("th", {"class": "aw"}).string
    home = soup.find("th", {"class": "hm"}).string
    date = soup.find_all('option', {'selected': True})
    date = date[2].string.replace('-', '')
    
    ### GET GAME excitement, comeback, mvp, lvp
    exc_text = 'Excitement</b></td><td>'
    exc_idx = score_data.index(exc_text)
    excitement_score = score_data[exc_idx + len(exc_text): exc_idx + len(exc_text) + 3]
    
    cb_text = 'Comeback</b></td><td>'
    cb_idx = score_data.index(cb_text)
    comeback_score = score_data[cb_idx + len(cb_text): cb_idx + len(cb_text) + 3]
    
    split = score_data.split('MVP</b></td><td>')[1]
    mvp = split.split('</td></tr>')[0]
    
    split = score_data.split('LVP</b></td><td>')[1]
    lvp = split.split('</td></tbody></table></div>')[0]
        
    ### GET EVENTS ###
    script = str(soup.findAll('script')[8])    
    split = script.split('"rows": ')[1]
    split = '{"rows": ' + split
    
    event_data = split.split('data_wprb.setColumnProperty')[0]
    event_data = event_data[:-23]
    events = []
    event_data = event_data.split('Q')
    for event in event_data:
        event = 'Q' + event
        event = event.replace('<u>', '').replace('</u>', '')
        event = event.replace('<b>', '').replace('</b>', '')
        event_split = event.split('<br />')
        if len(event_split) > 5:
            winprob_swing = event_split[2][-5:].lstrip()
            event = (event_split[1], winprob_swing)
            events.append(event)

    processed_game = "{},{},{},{},{},{},{},{}".format(home, away, date, excitement_score, comeback_score, mvp, lvp, events)
    return processed_game
    
def process_year(year, gameid):

    file = open('data/'+year+'/'+year+'_play_by_play.csv', 'a')
    
    game_number = 1
    for game in range(1,991):
        try:
            game_data = scrape(gameid, game)
            file.write(game_data)
            sleep(2)
            file.write('\n')
        except KeyError as e:
            print("key error:", e)
            continue
        except IndexError as e:
            print("index error:", e)
            continue
        
    file.close()

process_year('2011', 21100000)
# years = ['2006', '2007', '2008', '2009']
# game_ids = [20600000, 20700000, 20800000, 20900000]
# for i in range(0, 4):
#     process_year(years[i], game_ids[i])

21100001
21100002
21100003
21100004
21100005
21100006
21100007
21100008
21100009
21100010
21100011
21100012
21100013
21100014
21100015
21100016
21100017
21100018
21100019
21100020
21100021
21100022
21100023
21100024
21100025
21100026
21100027
21100028
21100029
21100030
21100031
21100032
21100033
21100034
21100035
21100036
21100037
21100038
21100039
21100040
21100041
21100042
21100043
21100044
21100045
21100046
21100047
21100048
21100049
21100050
21100051
21100052
21100053
21100054
21100055
21100056
21100057
21100058
21100059
21100060
21100061
21100062
21100063
21100064
21100065
21100066
21100067
21100068
21100069
21100070
21100071
21100072
21100073
21100074
21100075
21100076
21100077
21100078
21100079
21100080
21100081
21100082
21100083
21100084
21100085
21100086
21100087
21100088
21100089
21100090
21100091
21100092
21100093
21100094
21100095
21100096
21100097
21100098
21100099
21100100
21100101
21100102
21100103
21100104
21100105
21100106
21100107
21100108
21100109
21100110
21100111
2

21100895
21100896
21100897
21100898
21100899
21100900
21100901
21100902
21100903
21100904
21100905
21100906
21100907
21100908
21100909
21100910
21100911
21100912
21100913
21100914
21100915
21100916
21100917
21100918
21100919
21100920
21100921
21100922
21100923
21100924
21100925
21100926
21100927
21100928
21100929
21100930
21100931
21100932
21100933
21100934
21100935
21100936
21100937
21100938
21100939
21100940
21100941
21100942
21100943
21100944
21100945
21100946
21100947
21100948
21100949
21100950
21100951
21100952
21100953
21100954
21100955
21100956
21100957
21100958
21100959
21100960
21100961
21100962
21100963
21100964
21100965
21100966
21100967
21100968
21100969
21100970
21100971
21100972
21100973
21100974
21100975
21100976
21100977
21100978
21100979
21100980
21100981
21100982
21100983
21100984
21100985
21100986
21100987
21100988
21100989
21100990
